In [1]:
!pip install cudaq

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.5/122.5 MB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 107.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 95.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.8/280.8 MB 3.1 MB/s eta 0:00:00
  Created wheel for cudaq: filename=cudaq-0.13.0-py3-none-any.whl size=7002 sha256=c249c6eeb389c6fae4ef510a157496161e8b0528b8db846bcb102fa9cea86638
  Stored in directory: /root/.cache/pip/wheels/fb/8a/10/ee10abd8723a2fbfd7962112962900ec48cbba4de11305846d
Successfully built cudaq


In [9]:
import numpy as np
import random
import cudaq
import time
try:
    import cupy as cp
    DEVICE = "GPU (CuPy)"
except ImportError:
    import numpy as cp
    DEVICE = "CPU (NumPy)"

try:
    cudaq.set_target("nvidia")
    print("Successfully set target to NVIDIA GPU.")
except:
    print("NVIDIA GPU target not found. Falling back to CPU.")
    
def bits_to_spins(bits):
    return np.array([1 if b == 0 else -1 for b in bits])
N = 20

Successfully set target to NVIDIA GPU.


In [11]:
def calculate_energy_fft(sequences):
    """
    LABS probleminin enerji fonksiyonu: E = sum_{k=1}^{N-1} (C_k)^2
    Wiener-Khinchin teoremi kullanılarak O(N log N) hızında hesaplanır.
    """
    if sequences.ndim == 1:
        sequences = sequences.reshape(1, -1)

    batch_size, N = sequences.shape
    # FFT hızı için en yakın 2'nin kuvvetine tamamla
    n_fft = 2**((2 * N - 1).bit_length())

    # Fourier Dönüşümü ile otokorelasyon hesabı
    f_seq = cp.fft.rfft(sequences, n=n_fft, axis=1)
    autocorr = cp.fft.irfft(f_seq * cp.conj(f_seq), n=n_fft, axis=1)

    # k=1'den N-1'e kadar olan kısımların kareleri toplamı (C_k^2)
    # cp.round kullanıyoruz çünkü FFT float sonuç döner, LABS tam sayıdır.
    energies = cp.sum(cp.round(autocorr[:, 1:N])**2, axis=1)
    return energies

# --- [STEP 3: PARALEL ADAY LİSTELİ TABU SEARCH (EDUCATION)] ---
def tabu_search_gpu(initial_seq, max_iters=30, tabu_tenure=7):
    """
    Yerel arama adımı. GPU'da tüm komşuları aynı anda değerlendirir.
    """
    N = len(initial_seq)
    current_seq = cp.array(initial_seq)
    current_energy = calculate_energy_fft(current_seq)[0]

    best_seq = current_seq.copy()
    best_energy = current_energy
    tabu_list = cp.zeros(N, dtype=cp.int32)

    for it in range(max_iters):
        # Tüm 1-bit flip komşuları tek bir matriste oluştur (N, N)
        neighbors = cp.tile(current_seq, (N, 1))
        diag_indices = cp.arange(N)
        neighbors[diag_indices, diag_indices] *= -1

        # Enerjileri toplu halde (batch) hesapla
        neighbor_energies = calculate_energy_fft(neighbors)

        # Tabu durumu ve Aspiration (En iyiyi geçiyorsa yasağı del)
        is_not_tabu = (tabu_list <= it)
        is_aspiration = (neighbor_energies < best_energy)
        mask = is_not_tabu | is_aspiration

        neighbor_energies[~mask] = cp.inf

        # En iyi hareketi seç
        best_move_idx = cp.argmin(neighbor_energies)
        current_seq = neighbors[best_move_idx]
        current_energy = neighbor_energies[best_move_idx]

        if current_energy < best_energy:
            best_energy = current_energy
            best_seq = current_seq.copy()

        # Hamleyi yasaklılar listesine ekle
        tabu_list[best_move_idx] = it + tabu_tenure

    return best_seq, float(best_energy)

# --- [STEP 4: ANA MEMETİK TABU SEARCH DÖNGÜSÜ] ---
def run_full_mts(N=64, pop_size=20, generations=50, p_mutate=0.2, initial_population = None):
    """
    Memetik algoritmanın ana döngüsü: Combine -> Mutate -> Educate
    """
    # Popülasyonu başlat (İleride buraya Kuantum Seeds eklenebilir)
    if initial_population is not None:
            # Arkadaşından gelen listeyi GPU matrisine çevir
            quantum_pop = cp.array(initial_population)

            # Eğer kuantumdan gelen dizi sayısı pop_size'dan küçükse geri kalanı rastgele doldur
            if len(quantum_pop) < pop_size:
                extra_count = pop_size - len(quantum_pop)
                extra_pop = cp.random.choice(cp.array([-1, 1]), (extra_count, N))
                population = cp.vstack([quantum_pop, extra_pop])
            else:
                # Eğer kuantumdan çok fazla dizi gelirse en iyilerini/ilklerini al
                population = quantum_pop[:pop_size]
    else:
            # Dışarıdan veri gelmezse tamamen rastgele başlat
        population = cp.random.choice(cp.array([-1, 1]), (pop_size, N))

    global_best_seq = None
    global_best_energy = float('inf')
    history = []

    start_time = time.time()

    for gen in range(generations):
        # 1. COMBINE (Crossover): 3 ebeveynden çoğunluk oyu
        idx = cp.random.choice(pop_size, 3, replace=False)
        child = cp.sign(cp.sum(population[idx], axis=0))
        child[child == 0] = 1

        # 2. MUTATE: Rastgele bit çevirme
        if cp.random.random() < p_mutate:
            m_point = cp.random.randint(0, N)
            child[m_point] *= -1

        # 3. EDUCATION: Tabu Search ile yerel iyileştirme
        improved_child, improved_energy = tabu_search_gpu(child)

        # 4. SELECTION: Popülasyondaki en kötünün yerine koy (Steady-state update)
        pop_energies = calculate_energy_fft(population)
        worst_idx = cp.argmax(pop_energies)

        if improved_energy < pop_energies[worst_idx]:
            population[worst_idx] = improved_child

        # Global Best Güncelleme
        if improved_energy < global_best_energy:
            global_best_energy = improved_energy
            global_best_seq = improved_child.copy()

        history.append(global_best_energy)
        if gen % 10 == 0:
            print(f"Gen {gen:03d} | En İyi Enerji: {global_best_energy:.1f}")

    total_duration = time.time() - start_time
    return global_best_seq, global_best_energy, total_duration, history
best_sol, best_en, duration, hist = run_full_mts(N=N, generations=100)

Gen 000 | En İyi Enerji: 46.0
Gen 010 | En İyi Enerji: 38.0
Gen 020 | En İyi Enerji: 34.0
Gen 030 | En İyi Enerji: 34.0
Gen 040 | En İyi Enerji: 26.0
Gen 050 | En İyi Enerji: 26.0
Gen 060 | En İyi Enerji: 26.0
Gen 070 | En İyi Enerji: 26.0
Gen 080 | En İyi Enerji: 26.0
Gen 090 | En İyi Enerji: 26.0


In [12]:
def labs_energy(spins):
    N = len(spins)
    E = 0
    for k in range(1, N):
        Ck = np.sum(spins[:N-k] * spins[k:])
        E += Ck * Ck
    return E


In [13]:
@cudaq.kernel
def labs_p_layer_ansatz(n: int,
                        gammas: list[float],
                        betas: list[float]):
    q = cudaq.qvector(n)
    p = len(gammas)

    # Hadamard: tüm bitstringler
    for i in range(n):
        h(q[i])

    for layer in range(p):
        gamma = gammas[layer]
        beta  = betas[layer]

        # --- k = 1 ---
        for i in range(n - 1):
            cx(q[i], q[i+1])
            rz(2.0 * gamma * 1.0, q[i+1])
            cx(q[i], q[i+1])

        # --- k = 2 ---
        for i in range(n - 2):
            cx(q[i], q[i+2])
            rz(2.0 * gamma * 0.7, q[i+2])
            cx(q[i], q[i+2])

        # --- k = 3 ---
        for i in range(n - 3):
            cx(q[i], q[i+3])
            rz(2.0 * gamma * 0.4, q[i+3])
            cx(q[i], q[i+3])

        # Mixer
        for i in range(n):
            rx(2.0 * beta, q[i])


In [14]:
def quantum_cost(gammas, betas, n, shots=300, q_frac=0.1):
    result = cudaq.sample(
        labs_p_layer_ansatz,
        n,
        gammas,
        betas,
        shots_count=shots
    )

    energies = []
    for bitstring, count in result.items():
        bits = [int(b) for b in bitstring]
        E = labs_energy(bits_to_spins(bits))
        energies.extend([E] * count)

    energies = np.array(energies)
    k = max(1, int(len(energies) * q_frac))
    return np.mean(np.sort(energies)[:k])


In [15]:
def optimize_first_layer(n):
    gamma_grid = np.linspace(0.2, 1.2, 8)
    beta_grid  = np.linspace(0.1, 0.8, 8)

    best_cost = np.inf
    best_g, best_b = None, None

    for g in gamma_grid:
        for b in beta_grid:
            cost = quantum_cost([g], [b], n)
            if cost < best_cost:
                best_cost = cost
                best_g, best_b = g, b

    return [best_g], [best_b]


In [16]:
def add_and_optimize_layer(gammas, betas, n):
    # yeni layer küçük başlar
    gammas = gammas + [gammas[-1] * 0.5]
    betas  = betas  + [betas[-1]  * 0.5]

    gamma_grid = np.linspace(0.05, 0.6, 6)
    beta_grid  = np.linspace(0.05, 0.6, 6)

    best_cost = quantum_cost(gammas, betas, n)

    for g in gamma_grid:
        for b in beta_grid:
            test_g = gammas[:-1] + [g]
            test_b = betas[:-1]  + [b]
            cost = quantum_cost(test_g, test_b, n)
            if cost < best_cost:
                best_cost = cost
                gammas[-1] = g
                betas[-1]  = b

    return gammas, betas


In [17]:
def optimize_angles(n, p_max):
    gammas, betas = optimize_first_layer(n)
    print(f"p=1 → gammas={gammas}, betas={betas}")

    for p in range(2, p_max + 1):
        gammas, betas = add_and_optimize_layer(gammas, betas, n)
        print(f"p={p} → gammas={gammas}, betas={betas}")

    return gammas, betas


In [18]:
def quantum_seeds_cudaq(n, gammas, betas, shots):
    result = cudaq.sample(
        labs_p_layer_ansatz,
        n,
        gammas,
        betas,
        shots_count=shots
    )

    seeds = []
    for bitstring, count in result.items():
        bits = [int(b) for b in bitstring]
        seeds.extend([bits] * count)
    return seeds


In [19]:
def random_seeds(n, m):
    return [[random.randint(0, 1) for _ in range(n)] for _ in range(m)]


In [20]:

def energy_stats(bitstrings):
    energies = [labs_energy(bits_to_spins(b)) for b in bitstrings]
    return {
        "avg": np.mean(energies),
        "std": np.std(energies),
        "min": np.min(energies)
    }


In [22]:
SHOTS = 600
P_MAX = 3

gammas, betas = optimize_angles(N, P_MAX)

# 2) quantum seed
q_seeds = quantum_seeds_cudaq(N, gammas, betas, SHOTS)

q_stats = energy_stats(q_seeds)

# Run MTS using the quantum seeds
best_seq, best_energy, final_pop, history = run_full_mts(
    N=N,
    pop_size=len(q_seeds),   # Match population size to our seeds
    generations=50,
    initial_population=q_seeds
)
# 6. REPORT RESULTS
print("\n--- Final Results ---")
print(f"Best Energy Found: {best_energy}")
print(f"Best Sequence: {best_seq}")


p=1 → gammas=[np.float64(1.2)], betas=[np.float64(0.2)]
p=2 → gammas=[np.float64(1.2), np.float64(0.6)], betas=[np.float64(0.2), np.float64(0.05)]
p=3 → gammas=[np.float64(1.2), np.float64(0.6), np.float64(0.6)], betas=[np.float64(0.2), np.float64(0.05), np.float64(0.6)]
Gen 000 | En İyi Enerji: 54.0
Gen 010 | En İyi Enerji: 38.0
Gen 020 | En İyi Enerji: 38.0
Gen 030 | En İyi Enerji: 38.0
Gen 040 | En İyi Enerji: 38.0

--- Final Results ---
Best Energy Found: 38.0
Best Sequence: [ 1 -1 -1  1 -1 -1  1 -1  1 -1  1  1  1 -1 -1  1  1  1  1  1]

=== RANDOM ===
Avg Energy: 186.49
Std Dev Energy: 87.33
Min Energy: 46
